In [ ]:

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from scripts import analysis as a

# Model Results and Comparisons

This notebook is used to analyze the results from the models. More information can be found in the paper that was published alongside this repository.

### 1. Load the Data

Change the directory to analyze different results from different runs. If you run the main python file you can do this using the file path returned from `run_analysis` however there may not be enough buildings with valuable data to view the visualizations properly. This pipeline should be used with the `example_results_from_paper.csv` or with a larger dataset inputed by the user and run with `main.ipynb`.

In [ ]:
# Change this directory to analyze different results from different runs.
data_file = "data/example_results_from_paper.csv"
summary_df = pd.read_csv(data_file)
summary_df.head()

### 2. Remove instances where the models did not perform well

Remove the buildings for which the models couldn't obtain good results. To see the results without outliers removed, comment out these lines of code.

In [ ]:
## === REMOVE OUTLIERS ACCORDING TO STD AND FITTING ERROR
summary_df = summary_df[summary_df["euler_RC_std"] < (1000 / 12)]
summary_df = summary_df[summary_df["balance_stderr"] < 0.009]
summary_df = summary_df[summary_df["euler_cost_mean"] < 25]
summary_df = summary_df[summary_df["balance_p_value"] < 0.05]

summary_df.head()

### 3. Visualize the Performance of the decay curve method

In [ ]:
g = sns.jointplot(x="decay_RC_cost", y="decay_RC_std", data=summary_df, color="grey",)
g.set_axis_labels(
    xlabel="Mean fitting error ($K^2$)", ylabel="Standard deviation for RC (h)"
)

### 4. Visualize Performance of the Energy Balance Method

In [ ]:
g = sns.jointplot(x="euler_cost", y="euler_RC_std", data=summary_df, color="grey",)
g.set_axis_labels(
    xlabel="Mean fitting error ($K^2$)", ylabel="Standard deviation in RC (h)"
)

# =======================
g = sns.jointplot(x="euler_cost", y="euler_RK_std", data=summary_df, color="grey",)
g.set_axis_labels(
    xlabel="Mean fitting error  ($K^2$)", ylabel="Standard deviation in RK ($K$)"
)

# ========================
g = sns.jointplot(x="euler_cost", y="euler_cost_mean", data=summary_df, color="grey",)
g.set_axis_labels(
    xlabel="Mean fitting error ($K^2$)", ylabel="Mean overfitting cost ($K^2$)"
)

# ========================
g = sns.jointplot(x="euler_RC_std", y="euler_RK_std", data=summary_df, color="grey",)
g.set_axis_labels(
    xlabel="Standard deviation for RC (h)", ylabel="Standard deviation for RK ($K$)"
)

### 5. Compare the results from the models

In [ ]:
fix, axes = plt.subplots(1, 2, figsize=(20, 10), sharey=True, sharex=True)

for tick in axes[0].yaxis.get_major_ticks():
    tick.label.set_fontsize(20)

for tick in axes[0].xaxis.get_major_ticks():
    tick.label.set_fontsize(20)

for tick in axes[1].xaxis.get_major_ticks():
    tick.label.set_fontsize(20)


def plot_ax(
    ax,
    x_val,
    y_val,
    x_axis,
    y_axis,
    title,
    x_lim=None,
    y_lim=None,
    xlabel_size=35,
    ylabel_size=35,
):
    mask = ~np.isnan(x_val) & ~np.isnan(y_val)
    line = stats.linregress(x_val[mask], y_val[mask])
    slope, intercept, r_value, p_value, std_err = line

    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(25)
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(25)

    ax.scatter(x_val, y_val, marker="o", color="grey")
    x = np.linspace(0, 350, 1000)
    ax.plot(x, x, color="purple", label="Perfect Result", ls="dashed", linewidth=3)
    ax.set_xlabel(x_axis, fontsize=xlabel_size)
    ax.set_ylabel(y_axis, fontsize=ylabel_size)
    if x_lim:
        plt.xlim(x_lim[0], x_lim[1])

    if y_lim:
        plt.ylim(y_lim[0], y_lim[1])
    ax.plot(x, intercept + slope * x, "orange", label="Fitted Line", linewidth=3)
    ax.grid()
    ax.legend(fontsize=xlabel_size)


plot_ax(
    ax=axes[0],
    x_val=summary_df["balance_RK"],
    y_val=summary_df["euler_RK_mean"],
    x_axis="RK from balance points ($K$)",
    y_axis="Mean RK from\nenergy balance method ($K$)",
    title="RK: FONI vs. Balance Point",
)

plot_ax(
    ax=axes[1],
    x_val=summary_df["decay_RC_mean"],
    y_val=summary_df["euler_RC_mean"],
    x_axis="Mean RC from decay curves (h)",
    y_axis="Mean RC from\nenergy balance method (h)",
    title="RC: FONI Method vs. Decay Curves",
)
plt.show()

### 6. Compare Population Means

In [ ]:
new_df = summary_df[["euler_RC_mean", "decay_RC_mean"]].dropna()
print(
    stats.ttest_ind(new_df["euler_RC_mean"], new_df["decay_RC_mean"], equal_var=False)
)
print(
    stats.ttest_ind(
        summary_df["euler_RK_mean"], summary_df["balance_RK"], equal_var=False
    )
)

### 7. Compare the Standard Deviations and Model Fitting Errors

In [ ]:
sns.set(font_scale=10)
fig, axes = plt.subplots(2, 2, figsize=(50, 40), sharey="row", sharex="row")
first = axes[0, 0]
second = axes[0, 1]
third = axes[1, 0]
fourth = axes[1, 1]

labelsize = 70
axislabelsize = 90
legenfontsize = 75

first.set_ylabel("Frequency", fontsize=axislabelsize)
first.set_xlabel("RC standard Deviation (h)", fontsize=axislabelsize)
second.set_xlabel("RC mean (h)", fontsize=axislabelsize)

# =================== Plot the first row, RC
for tick in first.yaxis.get_major_ticks():
    tick.label.set_fontsize(labelsize)

for tick in first.xaxis.get_major_ticks():
    tick.label.set_fontsize(labelsize)

for tick in second.xaxis.get_major_ticks():
    tick.label.set_fontsize(labelsize)

summary_df["euler_RC_std"].hist(bins=25, ax=first)
summary_df["decay_RC_std"].hist(bins=25, ax=first)
first.legend(["Energy balance", "Decay curves"], prop={"size": legenfontsize})


summary_df["euler_RC_mean"].hist(bins=25, ax=second)
summary_df["decay_RC_mean"].hist(bins=25, ax=second)
second.legend(["Energy balance", "Decay curves"], prop={"size": legenfontsize})


# ===================== Plot the second row, RK
third.set_ylabel("Frequency", fontsize=axislabelsize)
third.set_xlabel(
    "RK standard deviation ($K$)/\nStandard error (0.0-1.0, scaled)",
    fontsize=legenfontsize,
)
fourth.set_xlabel("RK mean ($K$)", fontsize=axislabelsize)

for tick in third.yaxis.get_major_ticks():
    tick.label.set_fontsize(labelsize)

for tick in third.xaxis.get_major_ticks():
    tick.label.set_fontsize(labelsize)

for tick in fourth.xaxis.get_major_ticks():
    tick.label.set_fontsize(labelsize)

summary_df["balance_stderr_mult"] = summary_df["balance_stderr"] * 10000

summary_df["euler_RK_std"].hist(bins=25, ax=third)
summary_df["balance_stderr_mult"].hist(bins=50, ax=third)
third.legend(["Energy balance", "Balance point plots"], prop={"size": legenfontsize})

summary_df["euler_RK_mean"].hist(bins=25, ax=fourth)
summary_df["balance_RK"].hist(bins=50, ax=fourth)
fourth.legend(["Energy balance", "Balance point plots"], prop={"size": legenfontsize})

plt.grid()
plt.tight_layout()
plt.show()

### 8. Find the proportionate difference in results

In [ ]:
summary_df["RC_diffs"] = (
    (summary_df["euler_RC_mean"] - summary_df["decay_RC_mean"])
    / summary_df["euler_RC_mean"]
) * 100
summary_df["RK_diffs"] = (
    (summary_df["euler_RK_mean"] - summary_df["balance_RK"])
    / summary_df["euler_RK_mean"]
) * 100

plt.figure(figsize=(15, 15))
sns.set(font_scale=1)
sns.violinplot(data=summary_df[["RC_diffs", "RK_diffs"]],)

plt.xticks([0, 1], ["RC", "RK"], fontsize=40)
plt.yticks(fontsize=25)
plt.ylabel("Difference between the models (%)", fontsize=35)

### 9. Show the amount of decay curves that were found for each building

In [ ]:
summary_df["decay_amnt"].plot(kind="hist", figsize=(7, 7), grid=True, fontsize=17)
plt.xlabel("Decay curves found", fontsize=17)
plt.ylabel("Frequency", fontsize=17)

### 10. Correlate with the metadata

The code is here for reference but the full metadata was not released for privacy reasons

In [ ]:
_, _, _, metadata_file, _ = a.get_paths(data_file)
metadata_df = pd.read_csv(metadata_file)

corr_vals = [
    "decay_RC_mean",
    "euler_RC_mean",
    "euler_RK_mean",
    "balance_RK",
    "Age of Home [years]",
    "Floor Area [ft2]",
    "Number of Floors",
]

val_names = [
    "Mean RC:\ndecay curves",
    "Mean RC:\nenergy balance",
    "Mean RK:\nenergy balance",
    "RK:\nbalance points",
    "Age of home",
    "Floor area",
    "Number\nof floors",
]

size = "Floor Area [ft2]"
age = "Age of Home [years]"

join_df = summary_df.join(metadata_df.set_index("filename"), on="filename")
join_df["euler_R"] = join_df["euler_RK_mean"] / join_df[size]

# TODO
join_df["area_per_floor"] = join_df[size] / join_df["Number of Floors"]


def plot_corr(df, size=10):
    """Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot"""

    plt.figure(figsize=(10, 8))
    corr = df.corr()
    sns.set(font_scale=1.4)
    sns.heatmap(
        corr,
        cmap="coolwarm",
        annot=True,
        xticklabels=val_names,
        yticklabels=val_names,
        annot_kws={"size": 17},
    )
    plt.tight_layout()


plot_corr(join_df[corr_vals])